## 基础使用

In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
import dotenv
from regex import search

dotenv.load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

# 1. 加载
loader = TextLoader(
    file_path="./assets/ai.txt",
    encoding="utf-8",
)

docs = loader.load()

# 2. 创建文本拆分器
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

splitter_docs = text_splitter.split_documents(docs)

# 3. 创建嵌入模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
from langchain_community.vectorstores import FAISS

# 向量数据库
db = FAISS.from_documents(splitter_docs, embedding_model)

# 获取检索器
retriever = db.as_retriever()

# 检索
docs = retriever.invoke(input="深度学习是什么？")

print(len(docs))

for doc in docs:
    print(f"-------------{doc}")

2
-------------page_content='【人工智能（AI）简介】

一、什么是 AI
AI（Artificial Intelligence，人工智能）是让计算机像人一样思考、学习和决策的一门科学与技术。
简单来说，就是让机器“能理解世界、能做判断、能自己学习”，而不仅仅是执行命令。

AI 的目标是模仿人类智能，包括感知、理解、学习、推理和表达等能力。

------------------------------------------------------------
二、AI 的核心能力与对应技术
1. 感知：语音识别、图像识别、计算机视觉
2. 理解：自然语言处理（NLP）
3. 学习：机器学习（Machine Learning）
4. 推理：深度学习、强化学习
5. 表达：聊天机器人、文本生成（LLM）

------------------------------------------------------------
三、AI 的主要分支
1. 机器学习（Machine Learning）
   - 让计算机通过数据自动学习规律。
   - 类型：监督学习、无监督学习、强化学习。

2. 深度学习（Deep Learning）
   - 机器学习的一个分支，利用神经网络模拟人脑。
   - 应用于 GPT、图像识别、语音助手等。

3. 自然语言处理（NLP）
   - 让机器理解和生成人类语言。
   - 应用：ChatGPT、翻译、客服、情感分析。

4. 计算机视觉（CV）
   - 让计算机“看懂”图像或视频。
   - 应用：人脸识别、自动驾驶、医学影像。

5. 强化学习（Reinforcement Learning）
   - 通过“奖励与惩罚”让AI自己学习策略。
   - GPT使用了RLHF（人类反馈强化学习）。

------------------------------------------------------------
四、AI 的三个层次
1. 弱人工智能（Narrow AI）——只做特定任务（如ChatGPT）
2. 强人工智能（General AI）——能像人一样通用思考（尚未实现）
3. 超人工智能（Super AI）——超越人类智能（理论阶段）' metadata={'source'

In [4]:
# 1.导入相关依赖
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# 2.定义文档
document_1 = Document(
    page_content="经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！",
)
document_2 = Document(
    page_content="基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。",
)
document_3 = Document(
    page_content="气候变化：承诺到2030年将温室气体排放量减少50%。",
)
document_4 = Document(
    page_content=" 医疗保健：降低处方药价格，扩大医疗保险覆盖范围。",
)
document_5 = Document(
    page_content="教育：提供免费的社区大学教育。。",
)
document_6 = Document(
    page_content="科技：增加对半导体产业的投资以减少对外国供应链的依赖。。",
)
document_7 = Document(
    page_content="外交政策：继续支持乌克兰对抗俄罗斯的侵略。",
)
document_8 = Document(
    page_content="枪支管制：呼吁国会通过更严格的枪支管制法律。",
)
document_9 = Document(
    page_content="移民改革：提出全面的移民改革方案。",
)
document_10 = Document(
    page_content="社会正义：承诺解决系统性种族歧视问题。",
)
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
# 3.创建向量存储
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)
# 4.将文档向量化，添加到向量数据库索引中，得到向量数据库对象
db = FAISS.from_documents(documents, embeddings)

默认检索器使用相似性搜索

In [5]:
retriever = db.as_retriever(search_kwargs={"k": 4})

docs = retriever.invoke("经济政策")
for i, doc in enumerate(docs):
    print(f"\n结果{i + 1}:\n {doc.page_content}\n")


结果1:
 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


结果2:
 科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


结果3:
 外交政策：继续支持乌克兰对抗俄罗斯的侵略。


结果4:
 基础设施：政府将投资1万亿美元用于修复道路、桥梁和宽带网络。



分数阈值查询

In [6]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

docs = retriever.invoke("经济政策")
for doc in docs:
    print(f" 内容: {doc.page_content}")

 内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！


MMR 搜索

In [7]:
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"fetch_k": 2}
)

docs = retriever.invoke("经济政策")

print(len(docs))

for doc in docs:
    print(f"内容: {doc.page_content}")

2
内容: 经济复苏：美国经济正在从疫情中强劲复苏，失业率降至历史低点。！
内容: 科技：增加对半导体产业的投资以减少对外国供应链的依赖。。


## 通过FAISS构建一个可搜索的向量索引数据库，结合RAG让LLM回答问题

In [38]:
from langchain_core.prompts import PromptTemplate
import os
import dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

dotenv.load_dotenv()

# 创建Prompt
prompt_template = """
请使用以下提供的文本内容来回答问题。仅使用提供的文本信息，如果文本中没有相关信息，请回答：“抱歉,提供的文本内容中没有这个信息”。
文本内容：
{context}

问题：
{question}

回答：
"
"""

prompt = PromptTemplate.from_template(prompt_template)

# 初始化模型
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

llm = ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 加载文档
loader = TextLoader("./assets/nanjing.txt", encoding="utf-8")
documents = loader.load()

# 分割文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

texts = text_splitter.split_documents(documents)

# 创建向量存储，存储到内存中
vectorstore = FAISS.from_documents(
    documents = texts,
    embedding=embedding_model
)

# 获取检索器
retriever = vectorstore.as_retriever()
docs = retriever.invoke(input="南京有什么好吃的？")

# 创建链
chain = prompt | llm

result = chain.invoke(input={"question": "南京有什么好吃的？", "context": docs})

print("\n回答:", result.content)


回答: 南京的特色美食包括：
1. 盐水鸭：色白味香、皮嫩肉酥，是南京的传统名菜。
2. 鸭血粉丝汤：口感鲜美、汤汁浓郁，是南京人的“灵魂早餐”。
3. 小笼包、牛肉锅贴、桂花糖芋苗：老城南的经典小吃。
